# CSP Strategy — Slim Notebook

Imports from `csp/` package. All logic lives in modules, tested with 216 tests.

**Structure:**
1. Imports & Config
2. Connect & Initialize
3. Portfolio Status
4. Scan Diagnostics
5. Run Strategy
6. Utilities

In [1]:
# Imports
from dotenv import load_dotenv
load_dotenv(override=True)

from csp.config import StrategyConfig
from csp.clients import AlpacaClientManager
from csp.data.vix import VixDataFetcher
from csp.data.greeks import GreeksCalculator
from csp.data.manager import DataManager
from csp.signals.scanner import StrategyScanner
from csp.signals.diagnostics import print_scan_diagnostics
from csp.trading.execution import ExecutionEngine
from csp.trading.risk import RiskManager
from csp.trading.metadata import StrategyMetadataStore
from csp.trading.loop import TradingLoop
from csp.trading.portfolio import print_portfolio_status
from csp.trading.liquidation import liquidate_all_holdings

print("Imports OK")

Imports OK


In [2]:
# Configuration — edit overrides here
config = StrategyConfig(
    # Uncomment/edit to override defaults:
    # ticker_universe=["AAPL", "MSFT", "GOOG"],
    # starting_cash=100_000,
    # num_tickers=5,
    paper_trading=True,
)

print(f"Config: {len(config.ticker_universe)} symbols, cash=${config.starting_cash:,}")
print(f"Paper trading: {config.paper_trading}")

Config: 501 symbols, cash=$1,000,000
Paper trading: True


## Connect to Alpaca & Initialize Components

In [3]:
# Alpaca + Data Layer
alpaca = AlpacaClientManager(paper=config.paper_trading)
vix_fetcher = VixDataFetcher()
greeks_calc = GreeksCalculator()
data_manager = DataManager(alpaca, config)

print(f"Alpaca connected (paper={alpaca.paper})")
print(f"VIX: {vix_fetcher.get_current_vix():.2f}")

Alpaca connected (paper=True)
VIX: 21.20


In [4]:
# Scanner + Execution + Risk + Metadata
scanner = StrategyScanner(
    config=config,
    equity_fetcher=data_manager.equity_fetcher,
    options_fetcher=data_manager.options_fetcher,
    greeks_calc=greeks_calc,
)
execution = ExecutionEngine(alpaca, config)
risk_manager = RiskManager(config)
metadata = StrategyMetadataStore(path="strategy_metadata.json")

print("Scanner, Execution, Risk, Metadata — OK")

Scanner, Execution, Risk, Metadata — OK


In [5]:
# Build TradingLoop
loop = TradingLoop(
    config=config,
    data_manager=data_manager,
    scanner=scanner,
    metadata_store=metadata,
    risk_manager=risk_manager,
    execution=execution,
    vix_fetcher=vix_fetcher,
    greeks_calc=greeks_calc,
    alpaca_manager=alpaca,
)
print("TradingLoop ready")

  Daily log: logs/2026-02-16.json
TradingLoop ready


## Portfolio Status

In [6]:
print_portfolio_status(alpaca)

Alpaca Account Information
Account status:              AccountStatus.ACTIVE
Cash available:              $1,007,370.85
Buying power (with margin):  $4,029,483.40
Portfolio value:             $1,007,370.85
Options trading level:       3
Trading blocked:             False

Current Positions: None

Open Orders: None

Recent Order History (last 7 days, 100 orders):
  Time                 Symbol               Side        Qty Type     Status          Avg Price
  --------------------------------------------------------------------------------------------------------------
  2026-02-16 18:16:57  STX260220P00380000   sell          2 limit    canceled              N/A
  2026-02-16 18:16:26  STX260220P00380000   sell          2 limit    canceled              N/A
  2026-02-16 18:15:55  MU260220P00365000    sell          2 limit    canceled              N/A
  2026-02-16 18:15:24  MU260220P00365000    sell          2 limit    canceled              N/A
  2026-02-16 18:14:52  MU260220P00365000    sel

## Scan Diagnostics

In [7]:
print_scan_diagnostics(scanner, config, alpaca)

Universe Scan
Alpaca cash:                               $1,007,370.85
Short position collateral:                 $0.00
Available capital (cash - collateral):      $1,007,370.85
Max position size (10.0%):                $100,737.09

Symbols scanned:                         501
Passed equity filter:                     27
Passed equity + options filter:            4

Passed equity filter (27):
  Symbol       Price      SMA8     SMA20     SMA50      BB20    RSI   Collateral  Opts
  ----------------------------------------------------------------------------------------
  ANET     $  141.59    137.38    138.33    132.87    145.16   48.2 $    14,159     0
  BA       $  242.96    240.22    242.03    227.40    248.47   43.5 $    24,296     0
  CBOE     $  275.11    273.64    271.36    261.40    275.95   55.2 $    27,511     0
  CF       $   94.66     94.46     92.52     84.54     95.10   56.4 $     9,466     0
  CPT      $  108.53    108.37    108.15    107.60    109.37   51.3 $    10,853   

## Run Strategy

In [8]:
# Run a single cycle to verify everything works
summary = await loop.run_cycle()
print(f"\nCycle summary: Exits={summary.get('exits', 0)}, Entries={summary.get('entries', 0)}")

  Market closed today (2026-02-16 is not a trading day)

  Alpaca cash:                    $1,007,370.85
  Short position collateral:      $        0.00
  Available capital:               $1,007,370.85
  Max position size (10.0%):     $  100,737.09
  Active positions:                           0

Cycle summary: Exits=0, Entries=0


In [ ]:
# Live Trading Loop
poll_interval, max_cycles = (60, 100)

if execution.paper:
    print("Paper trading mode confirmed")
else:
    print("WARNING: LIVE TRADING MODE")
    print("Switch to paper trading before running the loop!")

await loop.run(poll_interval, max_cycles)

## Utilities

In [ ]:
# Uncomment to liquidate all holdings:
# liquidate_all_holdings(alpaca)